*Preparacion y limpieza de datos*

Pasamos a preparar y limpiar los datos. Para eso nos conectamos a la base de datos y cargaremos en un dataframe la informacion de la tabla oscar que creamos anteriormente

In [1]:
#importamos todas las bibliotecas necesarias
import sqlite3 as sq3
import pandas as pd
import numpy as np

In [2]:
#Creamos las funciones para conectar y desconectar de la base de datos
# CONEXIÓN
def get_connection():
    conn = sq3.connect('netflix.db')
    cur = conn.cursor()
    return conn, cur

# DESCONEXIÓN
def close_connection(conn, cur):
    cur.close()
    conn.close()

In [5]:
# Conectar a la base de datos
conn, cur = get_connection()

# Crear un DataFrame a partir de la tabla 'oscar'
query = "SELECT * FROM oscar;"
df_netflix_oscar = pd.read_sql_query(query, conn)

# Cerrar la conexión
close_connection(conn, cur)

# Mostrar las primeras filas del DataFrame
df_netflix_oscar.head()

,id_production,id_content,title_production,genre,premiere,runtime,imdb_score,language,type,title_content,...,based_on,starring,distributed_by,budget,budget_x_million,box_office,box_office_x_million,imdb,metascore,rotten_tomatoes
0,1.0,1.0,Dick Johnson Is Dead,Documentary,"October 2, 2020",90.0,7.5,English,Movie,Dick Johnson Is Dead,...,None,['George MacKay'. 'Dean-Charles Chapman'. 'Mar...,['Universal Pictures (Worldwide)'. 'Entertainm...,$95 million,95.0,$384.9 million,384.9,8.3,78.0,89.0
1,2.0,1093.0,Dolly Parton: A MusiCares Tribute,Documentary,"April 7, 2021",55.0,6.5,English,TV Show,Blood & Water,...,None,['George MacKay'. 'Dean-Charles Chapman'. 'Mar...,['Universal Pictures (Worldwide)'. 'Entertainm...,$95 million,95.0,$384.9 million,384.9,8.3,78.0,89.0
2,3.0,1071.0,Why Did You Kill Me?,Documentary,"April 14, 2021",83.0,5.6,English,TV Show,Ganglands,...,None,['George MacKay'. 'Dean-Charles Chapman'. 'Mar...,['Universal Pictures (Worldwide)'. 'Entertainm...,$95 million,95.0,$384.9 million,384.9,8.3,78.0,89.0
3,4.0,142.0,Extraction,Action,"April 24, 2020",117.0,6.7,English,TV Show,Jailbirds New Orleans,...,None,['George MacKay'. 'Dean-Charles Chapman'. 'Mar...,['Universal Pictures (Worldwide)'. 'Entertainm...,$95 million,95.0,$384.9 million,384.9,8.3,78.0,89.0
4,5.0,3988.0,Antoine Griezmann: The Making of a Legend,Documentary,"March 21, 2019",60.0,6.5,French,TV Show,Kota Factory,...,None,['George MacKay'. 'Dean-Charles Chapman'. 'Mar...,['Universal Pictures (Worldwide)'. 'Entertainm...,$95 million,95.0,$384.9 million,384.9,8.3,78.0,89.0


Ahora que ya creamos nuestro dataframe, vamos a evaluar toda su informacion para decidir que es necesario para preparar los datos para analisis

In [6]:
df_netflix_oscar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8747 entries, 0 to 8746
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id_production         551 non-null    float64
 1   id_content            551 non-null    float64
 2   title_production      551 non-null    object 
 3   genre                 551 non-null    object 
 4   premiere              551 non-null    object 
 5   runtime               551 non-null    float64
 6   imdb_score            551 non-null    float64
 7   language              551 non-null    object 
 8   type                  8747 non-null   object 
 9   title_content         8747 non-null   object 
 10  director              6122 non-null   object 
 11  cast                  7933 non-null   object 
 12  country               7923 non-null   object 
 13  date_added            8737 non-null   object 
 14  release_year          8747 non-null   int64  
 15  rating               

Paso a eliminar dupplicados utilizando como parametro la columna id_content, pues fue la columna que era comun en las 3 tablas

In [8]:
# Eliminar duplicados conservando la primera ocurrencia de 'id_content'
df_netflix_oscar_sin_duplicados = df_netflix_oscar.drop_duplicates(subset=['id_content'], keep='first')


Paso a averiguar los valores nulos en todo el dataframe por columna

In [14]:
num_filas_len = len(df_netflix_oscar_sin_duplicados)
print(num_filas_len)

552


In [13]:
# Verificar valores nulos por columna
valores_nulos_por_columna = df_netflix_oscar_sin_duplicados.isnull().sum(axis=0)

# Mostrar valores nulos por columna
print("Valores nulos por columna:")
print(valores_nulos_por_columna)

Valores nulos por columna:
id_production             1
id_content                1
title_production          1
genre                     1
premiere                  1
runtime                   1
imdb_score                1
language                  1
type                      0
title_content             0
director                188
cast                     62
country                 192
date_added                0
release_year              0
rating                    0
duration                  0
listed_in                 0
description               0
id_oscar                  0
id_inf                    0
id_res                    0
title_oscar               0
year_ceremony             0
category                  0
name                      0
winner                    0
directed_by               1
based_on                265
starring                 23
distributed_by           15
budget                   32
budget_x_million         32
box_office               22
box_office_x_million 

Si bien mis columnas principales no tienen tantos valores nulos, director, cast y country por ejemplo si lo tienen, por lo que no sería lo más apropiado borrar estos registros. decido entonces sustituir los valores nulos por 'NA'

In [16]:
# Sustituir valores nulos en columnas específicas con 'NA'
columnas_a_sustituir = ['director', 'cast', 'country']
df_netflix_oscar_sustituido = df_netflix_oscar_sin_duplicados.fillna(value='NA')



volvemos a averiguar los valores nulos

In [17]:
# Verificar valores nulos por columna
valores_nulos_por_columna = df_netflix_oscar_sustituido.isnull().sum(axis=0)

# Mostrar valores nulos por columna
print("Valores nulos por columna:")
print(valores_nulos_por_columna)

Valores nulos por columna:
id_production           0
id_content              0
title_production        0
genre                   0
premiere                0
runtime                 0
imdb_score              0
language                0
type                    0
title_content           0
director                0
cast                    0
country                 0
date_added              0
release_year            0
rating                  0
duration                0
listed_in               0
description             0
id_oscar                0
id_inf                  0
id_res                  0
title_oscar             0
year_ceremony           0
category                0
name                    0
winner                  0
directed_by             0
based_on                0
starring                0
distributed_by          0
budget                  0
budget_x_million        0
box_office              0
box_office_x_million    0
imdb                    0
metascore               0
rotten_toma

Con el dataframe ya preparado, pasamos a exportar a csv. será el archivo a ser usado en otro Notebook para analysis y también a ser cargado en Tableau y/o Looker Studio para visualizacion 

In [18]:
df_NO = df_netflix_oscar_sustituido

In [19]:
# Especificar el nombre del archivo CSV de salida
nombre_archivo_csv = 'netflix_oscar_cleaned.csv'

# Guardar el DataFrame en un archivo CSV
df_NO.to_csv(nombre_archivo_csv, index=False)

print(f"DataFrame guardado en {nombre_archivo_csv}")

DataFrame guardado en netflix_oscar_cleaned.csv
